In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from profile_generation import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook

In [4]:
input_file = 'Retrospective Liver Transplant Data.xlsx'
rows_to_skip = 17

# Get list of patients/sheet names
patient_list = get_sheet_names(input_file)

# Create and clean patient dataframe
patient_df = clean_data(patient_list, input_file, rows_to_skip)
patient_df



,day,response,dose,patient
0,0.0,NaN,0.0,84
1,1.0,NaN,0.0,84
2,2.0,2.4,0.5,84
3,3.0,2.8,1.0,84
4,4.0,3.2,1.5,84
...,...,...,...,...
28,28.0,NaN,0.0,138
29,29.0,NaN,0.0,138
30,30.0,NaN,0.0,138
31,NaN,*Note: Please avoid leaving the cell blank.\nE...,0.0,138
